In [1]:
from pydoc import importfile
import rdflib
import random
import pandas as pd
from sklearn.cluster import KMeans

from rdflib import ConjunctiveGraph
import gc, copy, datetime
#from pathlib import Path

import os, sys
p = os.path.abspath('..')
sys.path.insert(1, p)

from knowledge_graph import *

from rdf2vec import RDF2VecTransformer

from lcs_rdf_graph import LCS

from rdf_graph_utils import rdf_to_plot, rdf_to_text


def rdflib_to_kg_nquads(rdflib_g, label_predicates=[]):
    # Iterate over quads, add s, p and o to graph and 2 edges (s-->p, p-->o)
    # all predicates in label_predicates get excluded
    print(rdflib_g)
    kg = KnowledgeGraph()
    for s, p, o, _ in rdflib_g.quads((None, None, None, None)):
        if p not in label_predicates:
            s_v, o_v = Vertex(str(s)), Vertex(str(o))
            p_v = Vertex(str(p), predicate=True)
            kg.add_vertex(s_v)
            kg.add_vertex(p_v)
            kg.add_vertex(o_v)
            kg.add_edge(s_v, p_v)
            kg.add_edge(p_v, o_v)

    return kg


def get_drug_name(graph, drug):
    query = """
               SELECT ?obj
               WHERE { ?s <http://www.w3.org/2000/01/rdf-schema#label> ?obj.
               """.replace("?s", "<%s>" % drug) + "}"

    query_res = graph.query(query)
    label = []
    for res in query_res:
        label.append(str(res.obj))

    return label


In [2]:
print(end='Loading data... ', flush=True)
g = ConjunctiveGraph()

g.parse('../../datasets/drugbank/drugbank.nq', format="nquads")
print('OK')
#
# # Extract all database drugs' URI
all_drugs_file = pd.read_csv('../../datasets/drugbank/all_drugs.tsv', sep='\t')
all_drugs = [rdflib.URIRef(x) for x in all_drugs_file['drug']]
#
# # Define irrelevant predicates
predicates = pd.read_csv('../../datasets/drugbank/bad_predicates.tsv', sep='\t')
predicates = [rdflib.URIRef(x) for x in predicates['predicate']]
#

stop_patterns = pd.read_csv('../../datasets/drugbank/stop_patterns.tsv', sep='\t')
stop_patterns = [x for x in stop_patterns['stopping_patterns']]

preds = pd.read_csv('../../datasets/drugbank/uninformative.tsv', sep='\t')
preds = [rdflib.URIRef(x) for x in preds['uninformative']]


FileNotFoundError: [Errno 2] No such file or directory: '../../outputs/drugbank/testvalerio\\output_test_.turtle'

In [ ]:
kg = rdflib_to_kg_nquads(g, label_predicates=predicates)

del g
gc.collect()

#
# # %%
# # estraggo un'istanza di knowledge graph per ogni drug presente in quello iniziale

kg_depth = 4

i = 0
j = 0

kv = []
drugs = []
graphs = []

for drug in all_drugs:
    try:
        gi = extract_instance(kg, drug, kg_depth)
        graphs.append(gi)
        drugs.append(drug)
        kv.append({'graph': gi, 'resource': drug})
        i += 1
    except Exception as e:
        j += 1

print('ok:' + str(i))
print('not imported: ' + str(j))

del kg
gc.collect()


In [ ]:
# # Embeddings
transformer = RDF2VecTransformer(wl=False, max_path_depth=4, vector_size=15, walks_per_graph=1)
# transformer = RDF2VecTransformer()
transformer.fit(graphs, drugs)
embeddings = transformer.transform(graphs, drugs)



In [ ]:
# STAMPO PER OGNI RISORSA IL CLUSTER A CUI CORRISPONDE, IN MODO DA POTERNE PRELEVARE PER IL MOMENTO
# DUE APPARTENENTI ALLO STESSO CLUSTER
kmeans = KMeans(n_clusters=250)
km = kmeans.fit(embeddings)
y_kmeans = kmeans.predict(embeddings)

print("clustering models created")

In [ ]:

from pathlib import Path


clusters = {i: [] for i in range(kmeans.n_clusters)}
i = 0
for j in y_kmeans:
    clusters[j].append(i)
    i += 1

# Save clusters obtained from kmeans
#file = open("../../datasets/drugbank/clusters_log/kmeans_125_" + str(datetime.datetime.now()), "a")
#file = Path("../../datasets/drugbank/clusters_log/kmeans_125_" + str(datetime.datetime.now())).mkdir(parents=True, exist_ok=True)
file = open("../../datasets/drugbank/clusters_log/kmeans_125_2", "a")
file.write(str(clusters))
file.close()


In [ ]:
# Print clusters and their dimension

for key in clusters.keys():
    print("cluster " + str(key) + " with dimension: " + str(len(clusters[key])))


In [ ]:
directory = "../../outputs/drugbank/oLast_NQ_NEW"
if not os.path.exists(directory):
    os.mkdir(directory)


# cluster da analizzare
# k = 22

# seleziono il cluster con il minor numero di elementi

k = min(clusters.keys(), key=lambda a: len(clusters[a]))

# seleziono il cluster con il minor numero totale di archi

# k = min(clusters.keys(), 
#        key=lambda key: sum(len(kg_to_rdflib(graphs[res], kg_depth)) for res in clusters[key]))

# seleziono il cluster con l'elemento che ha il minor numero di archi

# k = min(clusters.keys(), key=lambda key: min([len(kg_to_rdflib(graphs[res], kg_depth))
#                                                 for res in clusters[key]]))


clusters[k] = sorted(clusters[k],  key=lambda res: len(kg_to_rdflib(graphs[res], kg_depth)))

# clusters[k] = [48, 50]

L = len(clusters[k])

print("esploro il cluster: " + str(k) + " con dimensione " + str(L))
print(clusters[k])
# risorsa iniziale
# resource_1 = random.choice(clusters[k])

resource_1 = clusters[k].pop(0)

graph_1 = graphs[resource_1]

print("risorsa iniziale: " + str(resource_1) + " di dim -> " + str(len(kg_to_rdflib(graph_1, kg_depth))))

# clusters[k].remove(resource_1)
explored_resoures = [resource_1]

iteration = 0



In [ ]:
# file = open(directory + "/tmp_LCS.log", "a")

while clusters[k]:
    # try:
        file = open(directory + "/tmp_LCS.log", "a")
        
        # resource_2 = random.choice(clusters[k])
   
        resource_2 = clusters[k].pop(0)
        graph_2 = graphs[resource_2]
        print("LCS with resource: " + str(resource_2) 
              + " con dim: " + str(len(kg_to_rdflib(graph_2, kg_depth))))
        # clusters[k].remove(resource_2)

        # seed
        seed = LCS(graph_1, graph_2, depth=2, stop_patterns=stop_patterns, uninformative_triples=preds)
        seed.find()
              
        explored_resoures.append(resource_2)
        
        print("dim LCS itermedio tra " + str(explored_resoures) + " \n--> " + str(len(seed)))
        
        del graph_1
        del graph_2
        gc.collect()
        
        graph_1 = copy.deepcopy(seed)

        print("Iterazione: " + str(iteration) + ", trovato LCS tra: " + str(explored_resoures))
        rdf_to_text(seed, directory, "turtle", "tmp_LCS_" + str(iteration))
        rdf_to_text(seed, directory, "nt", "tmp_LCS_" + str(iteration))
        
        del seed
        gc.collect()
        
        file.write("Iterazione: " + str(iteration) +  " risorse esplorate: " + str(explored_resoures) + "\n")
        file.close()
        
        iteration += 1
        """
        except Exception as e:
            print("got exception: " + str(e))
            file.close()
        """
# file.close()



In [ ]:
rdf_to_plot(graph_1, directory)
print("LCS finale tra le risorse: " + str(explored_resoures))

In [ ]:
# SALVATAGGIO DEL GRAFO

rdf_to_text(graph_1, directory, 'turtle', 'final_LCS')
rdf_to_text(graph_1, directory, 'nt', 'final_LCS')
file =  open(directory + "/final_LCS_resources.log", "a")
file.write(str(explored_resoures))

In [ ]:
print(end='Loading data... ', flush=True)
g = ConjunctiveGraph()

g.parse('../../datasets/drugbank/drugbank.nq', format="nquads")
print('OK')


In [ ]:
file.write("\n ####### \n")

explored_resources_names = []

for drug_index in explored_resoures:
    drug_name = get_drug_name(g, drugs[drug_index])
    explored_resources_names.extend(drug_name)

file.write(str(explored_resources_names))
file.close()

In [ ]:
del g
gc.collect()





